In [ ]:
import ee
import geemap
import os

In [ ]:
Map = geemap.Map()
Map

In [ ]:
site = ee.FeatureCollection('users/Introduction/study_area')
Map.addLayer(site, {}, "study_area")

In [ ]:
# Calculate Long Term Rainfall Average

# CHIRPS data comes as Pentad
chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/PENTAD")

# Long Period Average (LPA) should by 30 years or more

# Center to site
Map.centerObject(site, 9)

# List of 30 years
lpaYears = ee.List.sequence(2020, 2020)
months = ee.List.sequence(1, 12)

# # Introducting calendarRange()
# julyImages = chirps
#   .filter(ee.Filter.calendarRange(7, 7, 'month'))
# print(julyImages)

# Define time range
startyear = 2018
endyear = 2020

# Set date in ee date format
startdate = ee.Date.fromYMD(startyear,1,1)
enddate = ee.Date.fromYMD(endyear+1,12,31)

 
# create list for years
years = range(startyear,endyear);
 
# make a list with months
months = range(1,13)

#Filter to select Precipitation band
PrecipitationChirps = chirps.filterDate(startdate, enddate).sort('system:time_start', False).select("precipitation")

# Map over the years and create a monthly totals collection

# calculate the monthly Sum
def MonthlySum(imageCollection):
    mylist = ee.List([])
    for y in years:
        for m in months:
            w = imageCollection.filter(ee.Filter.calendarRange(y, y, 'year')).filter(ee.Filter.calendarRange(m, m, 'month')).sum()
            #mylist = mylist.add(w)
            mylist = mylist.add(w.set('year', y).set('month', m).set('date', ee.Date.fromYMD(y,m,1)).set('system:time_start',ee.Date.fromYMD(y,m,1)))
    return ee.ImageCollection.fromImages(mylist)


# run the calcMonthlyMean function
monthlyChirps = ee.ImageCollection(MonthlySum(PrecipitationChirps))


print("coll", monthlyChirps.first().getInfo())






In [26]:
image_ids = monthlyChirps.aggregate_array('system:index').getInfo()
print('Total images: ', len(image_ids))
print(monthlyChirps.first().getInfo())

Total images:  24
{'type': 'Image', 'bands': [{'id': 'precipitation', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'date': {'type': 'Date', 'value': 1514764800000}, 'system:time_start': {'type': 'Date', 'value': 1514764800000}, 'month': 1, 'year': 2018, 'system:index': '0'}}


In [27]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads/rainfall')
geemap.ee_export_image_collection(monthlyChirps, out_dir = out_dir, scale = 5000, region= site.geometry(), file_per_band= False)

TypeError: ee_export_image_collection() got an unexpected keyword argument 'filename'